In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/YOLOv7_pose/yolov7-pose-estimation

/content/gdrive/MyDrive/YOLOv7_pose/yolov7-pose-estimation


In [3]:
import numpy as np
import pandas as pd
from sklearn import tree, svm
from sklearn.metrics import accuracy_score, classification_report, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

In [4]:
df = pd.read_csv("test.csv")

In [5]:
columnName = []
for i in range(17):
  columnName.append("kp" + str(i) + "_X")
  columnName.append("kp" + str(i) + "_Y")
  columnName.append("kp" + str(i) + "_Confi")

In [6]:
df_cheating = df[df['label'] == 'Cheating']
df_noncheating = df[df['label'] == 'Non-Cheating']

In [7]:
df_cheating

,kp0_X,kp0_Y,kp0_Confi,kp1_X,kp1_Y,kp1_Confi,kp2_X,kp2_Y,kp2_Confi,kp3_X,...,kp14_X,kp14_Y,kp14_Confi,kp15_X,kp15_Y,kp15_Confi,kp16_X,kp16_Y,kp16_Confi,label
1506,0.617659,0.839054,0.863150,0.684358,0.875559,0.844554,0.563378,0.886501,0.764939,0.798646,...,0.0,0.013405,0.792924,0.0,0.0,0.0,0.0,0.0,0.0,Cheating
1507,0.618699,0.840609,0.875331,0.684703,0.878015,0.855298,0.564693,0.888477,0.785752,0.796680,...,0.0,0.008516,0.796468,0.0,0.0,0.0,0.0,0.0,0.0,Cheating
1508,0.622940,0.843531,0.846056,0.687709,0.880058,0.815234,0.569388,0.889436,0.746726,0.797397,...,0.0,0.023587,0.795436,0.0,0.0,0.0,0.0,0.0,0.0,Cheating
1509,0.620980,0.836386,0.881042,0.687272,0.873766,0.862819,0.566025,0.883769,0.797772,0.798552,...,0.0,0.011816,0.787432,0.0,0.0,0.0,0.0,0.0,0.0,Cheating
1510,0.619537,0.838653,0.887448,0.685610,0.876426,0.871103,0.564638,0.886251,0.806280,0.797854,...,0.0,0.019424,0.794113,0.0,0.0,0.0,0.0,0.0,0.0,Cheating
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6540,0.889130,0.945496,0.671690,0.000000,0.000000,0.000000,0.875501,1.000000,0.842320,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,Cheating
6541,0.893549,0.945435,0.662186,0.000000,0.000000,0.000000,0.880798,1.000000,0.836872,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,Cheating
6542,0.878806,0.945432,0.662122,0.000000,0.000000,0.000000,0.866325,1.000000,0.834942,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,Cheating
6543,0.870278,0.945342,0.687786,0.000000,0.000000,0.000000,0.856420,1.000000,0.851621,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,Cheating


In [8]:
df_noncheating

,kp0_X,kp0_Y,kp0_Confi,kp1_X,kp1_Y,kp1_Confi,kp2_X,kp2_Y,kp2_Confi,kp3_X,...,kp14_X,kp14_Y,kp14_Confi,kp15_X,kp15_Y,kp15_Confi,kp16_X,kp16_Y,kp16_Confi,label
0,0.627783,0.856736,0.923219,0.702739,0.880992,0.897338,0.575966,0.895504,0.880142,0.823285,...,0.000000,0.304594,0.866050,0.668223,0.052104,0.500082,0.053231,0.000000,0.514520,Non-Cheating
1,0.630523,0.858777,0.909266,0.703706,0.883392,0.874186,0.579122,0.896659,0.861140,0.820322,...,0.000000,0.304502,0.865908,0.000000,0.000000,0.000000,0.054519,0.000000,0.511976,Non-Cheating
2,0.628696,0.858738,0.928549,0.702393,0.883721,0.903198,0.576931,0.897197,0.889852,0.820132,...,0.000000,0.307547,0.863121,0.000000,0.000000,0.000000,0.051829,0.000000,0.503234,Non-Cheating
3,0.625742,0.857248,0.931711,0.699985,0.882158,0.912199,0.572713,0.895703,0.892731,0.820457,...,0.000000,0.303982,0.866609,0.000000,0.000000,0.000000,0.053558,0.000000,0.510904,Non-Cheating
4,0.626030,0.857502,0.921093,0.699691,0.882271,0.895631,0.573013,0.895215,0.878399,0.817954,...,0.000000,0.300596,0.867966,0.000000,0.000000,0.000000,0.051245,0.000000,0.512254,Non-Cheating
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5936,0.409495,0.858140,0.891660,0.479993,0.895990,0.927850,0.342490,0.902327,0.786151,0.647598,...,0.118642,0.243210,0.887838,0.790974,0.000000,0.686193,0.404293,0.039333,0.687370,Non-Cheating
5937,0.408809,0.858281,0.889903,0.477956,0.896134,0.925658,0.342450,0.902025,0.779671,0.645722,...,0.106124,0.242601,0.887305,0.793491,0.000000,0.687415,0.405893,0.038556,0.688159,Non-Cheating
5938,0.411675,0.859023,0.878882,0.481305,0.896417,0.915483,0.345015,0.902216,0.757801,0.649042,...,0.106038,0.257244,0.898706,0.791096,0.000000,0.719262,0.393406,0.051535,0.719531,Non-Cheating
5939,0.407110,0.859503,0.875994,0.477513,0.896373,0.915189,0.341427,0.903685,0.754751,0.648741,...,0.134676,0.246024,0.899425,0.793454,0.000000,0.723523,0.421999,0.043532,0.724942,Non-Cheating


input shape(num_shape, num_frame"num_time_step", num_joints,num_demention)= (1000,10,17,3)

In [9]:
no_of_timesteps = 10
num_joints = 17
num_demention = 3
n_sample = len(df_cheating)
df_cheating = np.resize(df_cheating[columnName],(int(n_sample/no_of_timesteps), no_of_timesteps, num_joints, num_demention))

n_sample = len(df_noncheating)
df_noncheating = np.resize(df_noncheating[columnName],(int(n_sample/no_of_timesteps), no_of_timesteps, num_joints, num_demention))

print(df_cheating)

[[[[0.61765935 0.83905355 0.8631503 ]
   [0.6843577  0.87555885 0.84455442]
   [0.56337843 0.88650052 0.76493919]
   ...
   [0.         0.01340521 0.7929244 ]
   [0.         0.         0.        ]
   [0.         0.         0.        ]]

  [[0.61869927 0.84060884 0.87533104]
   [0.68470312 0.87801517 0.85529798]
   [0.56469347 0.88847709 0.785752  ]
   ...
   [0.         0.0085162  0.79646838]
   [0.         0.         0.        ]
   [0.         0.         0.        ]]

  [[0.62293992 0.84353055 0.8460564 ]
   [0.68770865 0.8800584  0.81523371]
   [0.56938764 0.88943575 0.74672621]
   ...
   [0.         0.02358732 0.79543608]
   [0.         0.         0.        ]
   [0.         0.         0.        ]]

  ...

  [[0.6291872  0.83906959 0.85043168]
   [0.69237352 0.87791573 0.8217777 ]
   [0.57453781 0.88531164 0.75591403]
   ...
   [0.         0.0011572  0.78311741]
   [0.         0.         0.        ]
   [0.         0.         0.        ]]

  [[0.62093054 0.83630226 0.85406137]
   [0.6

In [11]:
X = np.append(df_cheating,df_noncheating , axis=0)

In [14]:
y = []
print(df_cheating.shape[0])
print(df_noncheating.shape[0])
for i in range(df_cheating.shape[0]):
  y.append(0)
for i in range(df_noncheating.shape[0]):
  y.append(1)
y = np.array(y)
print(y)

353
300
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 

In [15]:
print(X.shape)
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

(653, 10, 17, 3)
(653,)


Train SVM

In [16]:
model = svm.LinearSVC()
model.fit(X_train, y_train)
prediction = model.predict(X_test)

ValueError: ignored

In [ ]:
print(f"Test Set Accuracy : {accuracy_score(y_test, prediction) * 100} %\n\n")
print(f"calssification report {classification_report(y_test, prediction)}")

Train Decision Tree

In [ ]:
model = tree.DecisionTreeClassifier()
model =  model.fit(X_train, y_train)
prediction =  model.predict(X_test)


In [ ]:
print(f"Test Set Accuracy : {accuracy_score(y_test, prediction) * 100} %\n\n")
print(f"calssification report {classification_report(y_test, prediction)}")

Test Set Accuracy : 98.5485103132162 %


calssification report               precision    recall  f1-score   support

           0       0.99      0.98      0.99       725
           1       0.98      0.99      0.98       584

    accuracy                           0.99      1309
   macro avg       0.98      0.99      0.99      1309
weighted avg       0.99      0.99      0.99      1309



Train Ramdom Forest

In [ ]:
model = XGBClassifier()
model.fit(X_train, y_train)
preds = model.predict(X_test)
score = mean_absolute_error(preds, y_test)
acc = accuracy_score(preds, y_test)
print(score)
print(acc)

In [ ]:
numrandom1 = np.random.uniform(-0.05, 0.05)
numrandom2 = np.random.uniform(-0.05, 0.05)
print(numrandom1)
print(numrandom2)

0.001663069222455127
-0.04128283361534647


Model LSTM

In [80]:
import numpy as np
import pandas as pd
import keras 
from keras.layers import LSTM, Dense,Dropout, 
from keras.models import Sequential
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

SyntaxError: ignored

In [32]:
print(X.shape)

(653, 10, 17, 3)


In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [46]:
print(X_train.shape)

(522, 10, 17, 3)


In [72]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = Sequential()

model.add(LSTM(units = 50, return_sequences = True, input_shape = (X.shape[2], X.shape[3])))
model.add(Dropout(0.2))



model.add(Dense(units = 1, activation="sigmoid"))
model.compile(optimizer="adam", metrics = ['accuracy'], loss = "binary_crossentropy")
model.summary()
# model.fit(X_train, y_train, epochs=20, batch_size=32,validation_data=(X_test, y_test))

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_78 (LSTM)              (None, 17, 50)            10800     
                                                                 
 dropout_59 (Dropout)        (None, 17, 50)            0         
                                                                 
 dense_15 (Dense)            (None, 17, 1)             51        
                                                                 
Total params: 10,851
Trainable params: 10,851
Non-trainable params: 0
_________________________________________________________________


In [48]:

model  = Sequential()
model.add(LSTM(units = 50, return_sequences = True, input_shape = (X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 50))
model.add(Dropout(0.2))
model.add(Dense(units = 1, activation="sigmoid"))
model.summary()

# model.fit(X_train, y_train, epochs=20, batch_size=32,validation_data=(X_test, y_test))

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_24 (LSTM)              (None, 10, 50)            13600     
                                                                 
 dropout_16 (Dropout)        (None, 10, 50)            0         
                                                                 
 lstm_25 (LSTM)              (None, 10, 50)            20200     
                                                                 
 dropout_17 (Dropout)        (None, 10, 50)            0         
                                                                 
 lstm_26 (LSTM)              (None, 10, 50)            20200     
                                                                 
 dropout_18 (Dropout)        (None, 10, 50)            0         
                                                                 
 lstm_27 (LSTM)              (None, 50)              

Model ConvLSTM

In [82]:
model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(7, 7),
                     input_shape=(None, 101, 101, 1), padding='same',  return_sequences=True, 
                     activation='tanh', recurrent_activation='hard_sigmoid',
                     kernel_initializer='glorot_uniform', unit_forget_bias=True, 
                     dropout=0.3, recurrent_dropout=0.3, go_backwards=True ))
model.add(BatchNormalization())

model.add(ConvLSTM2D(filters=32, kernel_size=(7, 7), padding='same', return_sequences=True, 
                     activation='tanh', recurrent_activation='hard_sigmoid', 
                     kernel_initializer='glorot_uniform', unit_forget_bias=True, 
                     dropout=0.4, recurrent_dropout=0.3, go_backwards=True ))
model.add(BatchNormalization())

model.add(ConvLSTM2D(filters=32, kernel_size=(7, 7), padding='same', return_sequences=True, 
                     activation='tanh', recurrent_activation='hard_sigmoid', 
                     kernel_initializer='glorot_uniform', unit_forget_bias=True, 
                     dropout=0.4, recurrent_dropout=0.3, go_backwards=True ))
model.add(BatchNormalization())


model.add(ConvLSTM2D(filters=32, kernel_size=(7, 7), padding='same', return_sequences=False, 
                     activation='tanh', recurrent_activation='hard_sigmoid', 
                     kernel_initializer='glorot_uniform', unit_forget_bias=True, 
                     dropout=0.4, recurrent_dropout=0.3, go_backwards=True ))
model.add(BatchNormalization())

model.add(Conv2D(filters=1, kernel_size=(1, 1),
               activation='sigmoid',
               padding='same', data_format='channels_last')) 
model.summary()

Model: "sequential_43"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d_17 (ConvLSTM2D)  (None, None, 101, 101, 6  815616   
                             4)                                  
                                                                 
 batch_normalization_3 (Batc  (None, None, 101, 101, 6  256      
 hNormalization)             4)                                  
                                                                 
 conv_lstm2d_18 (ConvLSTM2D)  (None, None, 101, 101, 3  602240   
                             2)                                  
                                                                 
 batch_normalization_4 (Batc  (None, None, 101, 101, 3  128      
 hNormalization)             2)                                  
                                                                 
 conv_lstm2d_19 (ConvLSTM2D)  (None, None, 101, 101, 

In [86]:
model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(3, 3),
                     input_shape=(None, 10, 17, 3), padding='same',  return_sequences=True))
model.summary()

Model: "sequential_47"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d_24 (ConvLSTM2D)  (None, None, 10, 17, 64)  154624   
                                                                 
Total params: 154,624
Trainable params: 154,624
Non-trainable params: 0
_________________________________________________________________


In [67]:
import tensorflow as tf
from tensorflow.keras.layers import Input, ConvLSTM2D, BatchNormalization, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model

#Define the input shape
input_shape =(10, 17, 3) # (None, num_frames, num_joinys, num_dimensions) 

input_layer = Input(shape = input_shape)

conv_lstm_layer1 = ConvLSTM2D(filters=16, kernel_size = (3,3), padding ='same', return_sequences = True)(input_layer)
conv_lstm_layer1 = BatchNormalization()(conv_lstm_layer1)

conv_lstm_layer2 = ConvLSTM2D(filters = 32, kernel_size = (3,3), padding ='same', return_sequences = False)(conv_lstm_layer1)
conv_lstm_layer2 = BatchNormalization()(conv_lstm_layer2)

flatten_layer = Flatten()(conv_lstm_layer2)
dense_layer = Dense(units = 256, activation ='relu')(flatten_layer)
output_layer = Dense(units = 1, activation ='sigmoid')

model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
# model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=32)

ValueError: ignored